<a href="https://colab.research.google.com/github/ahmedtarek-/MNS_CP_2022/blob/main/stochastic_channel/Stochastic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from random import uniform

class StochasticKChannel:
  alpha_n = 0.044
  beta_n = 0.02

  def __init__(self, initial_state):
    self.state = initial_state

  def state_1_transition(self):
    random_1 = uniform(0, 1)
    if random_1 < 4*self.alpha_n:
      self.state = 2
      return True

  def state_2_transition(self):
    """
      Probability to leave: 3*alpha_n + beta_n
      Probability to go to S3: 3*alpha_n / (3*alpha_n + beta_n)
      Probability to go to S1: 1- (3*alpha_n / (3*alpha_n + beta_n))
    """
    random_1 = uniform(0, 1)
    random_2 = uniform(0, 1)
    p_leave = 3*self.alpha_n + self.beta_n
    p_s3 = 3*self.alpha_n / (3*self.alpha_n + self.beta_n)
    if random_1 > p_leave:
      return False  # Stay in state 1
    if random_2 <= p_s3:
      self.state = 3
    else:
      self.state = 1
    return True

  def state_3_transition(self):
    """
      Probability to leave: 2*alpha_n + 2*beta_n
      Probability to go to S3: 3*alpha_n / (3*alpha_n + beta_n)
      Probability to go to S1: 1- (3*alpha_n / (3*alpha_n + beta_n))
    """
    random_1 = uniform(0, 1)
    random_2 = uniform(0, 1)
    p_leave = 3*alpha_n + beta_n
    p_s3 = 3*alpha_n / (3*alpha_n + beta_n)
    if random_1 > p_leave:
      return False  # Stay in state 1
    if random_2 <= p_s3:
      self.state = 3
    else:
      self.state = 1
    return True
    

  def next_state(self):
    if self.state == 1:
      self.state_1_transition()
    elif self.state ==  2:
      self.state_2_transition()
    elif self.state == 3:
      self.state_3_transition()
    elif self.state == 4:
      self.state_4_transition()
    elif self.state ==  5:
      self.state_5_transition()

channel = StochasticKChannel(1)
print("initial: {}".format(channel.state))
channel.next_state()
print("after one move: {}".format(channel.state))

initial: 1
after one move: 1
